In [ ]:
TODO  18、19

订单
洗数据
51按天订单量
51按小时订单量








In [ ]:
TO do
done 1. 缺失数据清理，不处理100的
2. 数据聚类做两个不一样的
a. 上下车坐标，做空间分布上的聚类，说明供求关系（暂缓）
b. 按照盈利能力分组，即做数据处理的时候，需要算出每单的收益（优先）




18年
计算公式如下：收入
白天：10元/3公里，之后每公里2元，单次8公里以上，加收50%空乘费用
晚上：23~5am,起步价12元，加收20%夜间补贴，3公里之后，每公里2.4元


接着按照BIRCH分组的目的是分出高利润和低利润2~3组（按情况而定），同时算出每组的平均收益和平均载客距离




接着对每组数据做如下分析：

首先，算出最高利润和最低利润组分别的前三的众数

聚类
    出发
    到达
    
    
接着，按照小时和第几天做上车次数统计（即按照上车坐标groupby），对于高利润组和低利润组分别分析

最后按照前3+3做基础，做后面的预测分析

In [ ]:
mp2std = df['trip_duration'].mean() + 2*df['trip_duration'].std()
mp2std

In [ ]:
import pandas as pd
import datetime

DATA_PATH = '/data/'
DATA_TAXI_ORDER = DATA_PATH+'taxiOrder/'
DATA_FILE_NAME = 'taxiOrder201951-54.csv'


df = pd.read_csv(DATA_TAXI_ORDER+DATA_FILE_NAME)
total_num = len(df)

df.dropna(subset=['GETON_LONGITUDE', 
                  'GETON_LATITUDE', 
                  'GETOFF_LONGITUDE', 
                  'GETOFF_LATITUDE',
                  'PASS_MILE',
                  'CAR_NO'], inplace=True)
na_num = len(df)

df['GETON_DATE'] = pd.to_datetime(df['GETON_DATE'])
df['GETOFF_DATE'] = pd.to_datetime(df['GETOFF_DATE'])

df['trip_duration'] = (df['GETOFF_DATE'] - df['GETON_DATE']).dt.total_seconds()/60
df = df[df['trip_duration'] > 1]
more_then_1_num = len(df)


# filter data 
lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]
df_filtered = df[
    (df['GETON_LONGITUDE'] > long_range[0])
    & (df['GETON_LONGITUDE'] < long_range[1])
    & (df['GETON_LATITUDE'] > lat_range[0])
    & (df['GETON_LATITUDE'] < lat_range[1])
    & (df['GETOFF_LONGITUDE'] > long_range[0])
    & (df['GETOFF_LONGITUDE'] < long_range[1])
    & (df['GETOFF_LATITUDE'] > lat_range[0])
    & (df['GETOFF_LATITUDE'] < lat_range[1])
    & (df['trip_duration'] > 0.5)
    & (df['trip_duration'] < 37)
#     & (df['WAITING_TIME'] < 120)
#     & (df['PASS_MILE'] > 0.1)
]
in_range_num = len(df)

# sorted(df_filtered['WAITING_TIME'].unique())[-10:]

df_filtered['trip_time'] = df['trip_duration'] - df['WAITING_TIME']/60
for key, item in df_filtered.groupby('trip_time'):
    print(key, len(item))


df_filtered['day'] = df_filtered['GETON_DATE'].apply(
    lambda dt: datetime.datetime(dt.year, dt.month, dt.day)
)

df_filtered['hour'] = df_filtered['GETON_DATE'].dt.hour
df_filtered['off_hour'] = df_filtered['GETOFF_DATE'].dt.hour


taxi_base_mile = 3
taxi_empty_return_mile = 8
taxi_wait_time = 75

base = [10,12]
pm = [2,2.4]
erpm = [3,3.6]

pwt = [0.5,0.6]
erpwt = [0.75,0.9]


def price(row):
    h = row[0]
    m = row[1]
    t = row[2]
    
    is_night = 0 if h < 23 and h > 5 else 1
    
    p = base[is_night]
    if (m-taxi_base_mile) > 0: # 大于3公里
        if (m-taxi_empty_return_mile) < 0: # 小于8公里
            # base + 3_8公里之间的 + 等待时间
            p = p + (m-taxi_base_mile)*pm[is_night] + t*pwt[is_night]
        else:
            # base + 3_8公里之间的 + 8公里之外的 + 等待时间
            p = p + (taxi_empty_return_mile-taxi_base_mile)*pm[is_night] 
            + (m-taxi_empty_return_mile)*erpm[is_night] + t*erpwt[is_night]
        
    return p
    

df_filtered['off_hour'] = df_filtered['GETOFF_DATE'].dt.hour
df_filtered['price'] = df_filtered[['off_hour', 'PASS_MILE', 'WAITING_TIME']].apply(price, axis=1)



In [ ]:
# KMean
from sklearn.cluster import KMeans
import numpy as np

km = KMeans(n_clusters=2, random_state=0).fit(np.array(df_filtered['price'].tolist()).reshape(-1,1))
km.labels_



In [ ]:
import numpy as np
from sklearn.cluster import Birch

l_l_long_lat = np.array(df_filtered[['GETON_LONGITUDE', 'GETON_LATITUDE']].tolist())
brc = Birch(branching_factor=50, n_clusters=None, threshold=0.0001)
brc.fit(l_l_long_lat)
labels = brc.predict(l_l_long_lat)

In [ ]:
# taxi_base = 10
# taxi_base_n = 12

# taxi_per_mile = 2
# taxi_empty_return_per_mile = 3

# taxi_per_mile_n = 2.4
# taxi_empty_return_per_mile_n = 3.6

# taxi_per_wait_time = 0.5
# taxi_empty_return_per_wait_time = 0.75

# taxi_per_wait_time_n = 0.6
# taxi_empty_return_per_wait_time_n = 0.9


taxi_base_mile = 3
taxi_empty_return_mile = 8
taxi_wait_time = 75

base = [10,12]
pm = [2,2.4]
erpm = [3,3.6]

pwt = [0.5,0.6]
erpwt = [0.75,0.9]



def price(row):
    h = row[0]
    m = row[1]
    t = row[2]
    
    is_night = 0 if h < 23 and h > 5 else 1
    
    p = base[is_night]
    if (m-taxi_base_mile) > 0: # 大于3公里
        if (m-taxi_empty_return_mile) < 0: # 小于8公里
            # base + 3_8公里之间的 + 等待时间
            p = p + (m-taxi_base_mile)*pm[is_night] + t*pwt[is_night]
        else:
            # base + 3_8公里之间的 + 8公里之外的 + 等待时间
            p = p + (taxi_empty_return_mile-taxi_base_mile)*pm[is_night] 
            + (m-taxi_empty_return_mile)*erpm[is_night] + t*erpwt[is_night]
        
    return p
    

df_filtered['off_hour'] = df_filtered['GETOFF_DATE'].dt.hour
df_filtered['price'] = df_filtered[['off_hour', 'PASS_MILE', 'WAITING_TIME']].apply(price, axis=1)



# 1. 订单数据

## 1.1 统计分析
 ⼯作⽇/⾮⼯作⽇：⽤巡游⻋的10.19~10.27处理，其中19，20，26，27为⾮⼯作⽇，其余为⼯作⽇，计算出⾏量，可考虑的第三变量为时间。

In [ ]:
# TODO 
# 1.nan
# 2.出厦门的订单

In [ ]:
import pandas as pd
import datetime

DATA_PATH = '/data/'
DATA_TAXI_ORDER = DATA_PATH+'taxiOrder/'
DATA_FILE_NAME = 'taxiOrder20191019-27.csv'


df = pd.read_csv(DATA_TAXI_ORDER+DATA_FILE_NAME)
df['GETON_DATE'] = pd.to_datetime(df['GETON_DATE'])

t_df = df
t_df['GETON_DATE'] = t_df['GETON_DATE'].apply(
    lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour)
)

for key,item in t_df.groupby('GETON_DATE'):
    print(key,item['GETON_LONGITUDE'].count())
# print(t_df.groupby('GETON_DATE')['GETON_LONGITUDE'].count())



TODO
2019年51数据，四天

 热⻔区域：通过上客点和下客点的分别聚类或者纯数学⽅法得出；
  聚类的话热⻔即是最多的class，纯数学⽅法则是按照点选择周围上下客点，圆圈聚类，然后过滤，只取top20
  


 平均速度：通过（载客距离+空载距离）/总时间得出
TODO 载客平均速度=载客距离/time，总平均速度=（载客距离+空载距离）/time，不载客平均速度=不宰客、。。。


2019年51数据，四天

 平均载客距离：通过载客距离之和/总订单数得出
TODO 洗一下数据，PASS_MILE，NON-

In [ ]:
import pandas as pd
import datetime

DATA_PATH = '/data/'
DATA_TAXI_ORDER = DATA_PATH+'taxiOrder/'
DATA_FILE_NAME = 'taxiOrder201951-54.csv'


df = pd.read_csv(DATA_TAXI_ORDER+DATA_FILE_NAME)
df['GETON_DATE'] = pd.to_datetime(df['GETON_DATE'])
df['GETOFF_DATE'] = pd.to_datetime(df['GETOFF_DATE'])

df['duration'] = df['GETOFF_DATE'] - df['GETON_DATE']
df['day'] = df['GETON_DATE'].apply(
    lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour)
)


# 计算每辆车，每天的，每小时的
list_carno = df['CAR_NO'].unique().tolist()

gp = df.groupby('CAR_NO')
t_ = None
for key, item in gp:
    t_df = item.groupby('day')
    for k, it in t_df:
        t_ = it[['PASS_MILE','duration']].sum()
        print(key, k, t_.iloc[0]/t_.iloc[1].total_seconds()*3600)



# TODO ，每天的平均速度，每小时的平均速度 
t_df = df.groupby('day')[['PASS_MILE','duration']]
print(t_df.dtypes)
print(t_df)
print(t_df.iloc[0]/(t_df.iloc[0]))



#51的平均速度
t_df = df[['PASS_MILE','duration']].sum()
#print(t_df.iloc[0]/(8389*24+4+18/60))


In [ ]:
# TODO write file
FILE_PATH = '/hm/'
FILE_NAME = FILE_PATH+'.txt'

out = open("myOutFile.txt", "w")
for line in xxx:
    out.write(line)
out.close()


In [ ]:
# 有可能存在，订单的时间不对的问题。（结束-开始）/mile，存在极大的问题。可以分析的包括，1.是不是有拥堵；2.是不是
# 起点，或者轨迹点，超出厦门地区的点
# 进阶版，只算轨迹在厦门内的，超出去的可以

In [ ]:
lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]

In [ ]:
import pandas as pd
import datetime

DATA_PATH = '/data/'
DATA_TAXI_ORDER = DATA_PATH+'taxiOrder/'
DATA_FILE_NAME = 'taxiOrder201951-54.csv'


df = pd.read_csv(DATA_TAXI_ORDER+DATA_FILE_NAME)
total_num = len(df)

df.dropna(subset=['GETON_LONGITUDE', 
                  'GETON_LATITUDE', 
                  'GETOFF_LONGITUDE', 
                  'GETOFF_LATITUDE',
                  'PASS_MILE',
                  'CAR_NO'], inplace=True)
na_num = len(df)

df['GETON_DATE'] = pd.to_datetime(df['GETON_DATE'])
df['GETOFF_DATE'] = pd.to_datetime(df['GETOFF_DATE'])

df['trip_duration'] = (df['GETOFF_DATE'] - df['GETON_DATE']).dt.total_seconds()/60
df = df[df['trip_duration'] > 1]
more_then_1_num = len(df)

# t_list = sorted(df['PASS_MILE'].unique().tolist())
# t_list[-1]

# df.conlumns
# for i in df[df['PASS_MILE'] == t_list[-1]].values:
#     print(i)

# t_list[:10]
# for i in range(0,10):
#     print(i, len(df[df['PASS_MILE'] == (i/10)]))


# filter data 
lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]
df_filtered = df[
    (df['GETON_LONGITUDE'] > long_range[0])
    & (df['GETON_LONGITUDE'] < long_range[1])
    & (df['GETON_LATITUDE'] > lat_range[0])
    & (df['GETON_LATITUDE'] < lat_range[1])
    & (df['GETOFF_LONGITUDE'] > long_range[0])
    & (df['GETOFF_LONGITUDE'] < long_range[1])
    & (df['GETOFF_LATITUDE'] > lat_range[0])
    & (df['GETOFF_LATITUDE'] < lat_range[1])
    & (df['trip_duration'] > 0.5)
    & (df['trip_duration'] < 37) # 34
#     & (df['trip_duration'] < 1)
#     & (df['PASS_MILE'] > 0.1)
]
in_range_num = len(df)

df_filtered['day'] = df_filtered['GETON_DATE'].apply(
    lambda dt: datetime.datetime(dt.year, dt.month, dt.day)
)

df_filtered['hour'] = df_filtered['GETON_DATE'].dt.hour
df_filtered['off_hour'] = df_filtered['GETOFF_DATE'].dt.hour

result = []
for key, item in df_filtered.groupby('day'):
    result.append(key)
    for k, it in item.groupby('hour'):
        totalmile = item['PASS_MILE'].sum()
        totaltime = item['trip_duration'].sum()/60
        result.append('day:{} 平均载客距离:{} 平均速度:{}'.format(k, totalmile/len(item), totalmile/totaltime))

        

FILE_PATH = '/hm/'
FILE_NAME = FILE_PATH+'2019-51-hour-ave-mile-speed.txt'

out = open(FILE_NAME, "w")
for line in result:
    out.write(line)
out.close()

    


#     print(' ')
  

for key, item in df_filtered.groupby('day'):
    print(' ')
    print(' ')
    print(key)
    for h,t in item.groupby('hour'):
        print(h, len(t))
#     input("Press Enter to continue...")




# t_list = sorted(df_filtered['PASS_MILE'].unique().tolist())
# t_list[-1]
# print('total_num:{} na_num:{} more_then_1_num:{} in_range_num:{}'.format(
#     total_num, na_num, more_then_1_num, in_range_num)
#      )


t_df = df_filtered[df_filtered['price'] == 21836.5]
print(len(t_df))
t_df.head()
for i in range(0,10):
    print(i, len(t_df))
    
    
    
from sklearn.cluster import KMeans
import numpy as np

km = KMeans(n_clusters=2, random_state=0).fit(np.array(df_filtered['price'].tolist()).reshape(-1,1))
df_filtered['price_cluster'] = km.labels_

for key, item in df_filtered.groupby('price_cluster'):
    totalprice = item['price'].sum()
    totalmile = item['PASS_MILE'].sum()
    totaltime = item['trip_duration'].sum()/60
    print('cluster:{} 总价:{} count:{} 均价:{}'.format(key, totalprice, len(item), totalprice/len(item)))
    print('cluster:{} 总公里:{} 总用时:{} 平均速度:{}'.format(key, totalmile, totaltime, totalmile/totaltime))
    print(' ')
    print(' ')
    



In [ ]:
# 网约车 最远距离订单
import pandas as pd
import datetime

DATA_PATH = '/data/'
DATA_TAXI_ORDER = DATA_PATH+'wycOrder/'
DATA_FILE_NAME = 'wycOrder20191019-1027.csv'


df = pd.read_csv(DATA_TAXI_ORDER+DATA_FILE_NAME).dropna(subset=['DEP_LONGITUDE', 'DEP_LATITUDE', 'DEST_LONGITUDE', 'DEST_LATITUDE'])
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'])
df['DEST_TIME'] = pd.to_datetime(df['DEST_TIME'])

df['trip_duration'] = (df['DEST_TIME'] - df['DEP_TIME']).dt.total_seconds()/60
t_list = sorted(df['DRIVE_MILE'].unique().tolist())
t_list[-1]


In [ ]:
df.loc[df['PASS_MILE'] == 1358.7]

In [ ]:
FILE_PATH = '/hm/'
FILE_NAME = FILE_PATH+'test.txt'

out = open(FILE_NAME, "w")
out.write(df[])

In [ ]:
pd.plotting.scatter_matrix(frame = df[['PASS_MILE', 'trip_duration']], figsize=(7,7));


In [ ]:
df_filtered = df[
]

In [ ]:
# 微博，打卡

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df['GETON_LATITUDE'],df['GETON_LONGITUDE'])
)

import geopandas as gpd
from shapely.geometry import Point

df_geometry = [Point(xy) for xy in zip(df.GETON_LONGITUDE, df.GETON_LATITUDE)]
gdf = gpd.GeoDataFrame(df, geometry=df_geometry)
